In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
np.random.seed(42) # 设置经典随机种子

simulation_time = 1000  # 总模拟时间
dt = 1.0  # 时间步长
time_steps = int(simulation_time / dt) # 模拟参数

tau_m = 10.0  # 膜时间常数
V_rest = 0.0  # 初始膜电位
V_reset = 0.0  # 重置电位
V_th = 0.5  # 阈值

A_plus = 0.051  # 突触增强幅度
A_minus = 0.050  # 突触减弱幅度
tau_plus = 20.0  # 突触增强时间常数
tau_minus = 20.0  # 突触减弱时间常数
w_max = 1.0  # 突触权重最大值
w_min = 0.2  # 突触权重最小值

num_inputs = 5 # 模拟神经元数量

In [3]:
def input_sequences():
    sequence = np.zeros((time_steps, num_inputs))
    spike_times = [0, 10, 20, 30, 40] # 输入脉冲的时间点
    repeats = 20
    interval = 50
    # 创建一个二维全零数组表示所有输入神经元都未发放脉冲
    # 随后定义一个固定的脉冲时间点 表示每个神经元强制输出脉冲的时间
    # 该组脉冲时间点在这组训练中会被重复20次 每次开始的间隔为50ms

    for r in range(repeats):
        for idx, t in enumerate(spike_times):
            time_idx = int((t + r * interval) / dt)
            if time_idx < time_steps:
                sequence[time_idx, idx] = 1
        # 计算出每个脉冲应该被强制输出的时间 记录到sequence数组中

    return sequence

In [4]:
class LIFNeuron:
    def __init__(self):
        self.V = V_rest
        self.spike = False

    def update(self, I):
        dV = (-self.V + I) / tau_m * dt
        self.V += dV
        if self.V >= V_th:
            self.spike = True
            self.V = V_reset
        else:
            self.spike = False
        # 如果达到了阈值 则发送脉冲并重置膜电位

In [5]:
class STDP:
    def __init__(self, num_pre, num_post):
        self.w = np.random.uniform(0.8, 1.0, (num_pre, num_post))  # 初始化权重
        self.tau_pre = tau_plus
        self.tau_post = tau_minus
        self.A_plus = A_plus
        self.A_minus = A_minus
        self.x_pre = np.zeros(num_pre)
        self.x_post = np.zeros(num_post)

    def update_weights(self, pre_spikes, post_spikes):
        self.x_pre *= np.exp(-dt / self.tau_pre)
        self.x_post *= np.exp(-dt / self.tau_post)
        # 对突触前神经元的时间踪迹和突触后神经元的时间踪迹进行指数衰减

        self.x_pre += pre_spikes
        self.x_post += post_spikes
        # 将上一步的时间踪迹和当前步进行累加

        dw = np.outer(self.A_plus * post_spikes, self.x_pre) - np.outer(self.x_post, self.A_minus * pre_spikes)
        # 第一部分突触增强由突触前神经元的时间踪迹和突触后脉冲决定
        # 第二部分突触减弱由突触后神经元的时间踪迹和突触前脉冲决定

        self.w += dw.T
        self.w = np.clip(self.w, w_min, w_max)
        # 将权重变化转置后加到当前突触权重的矩阵上
        # 并通过剪裁确保突触权重在设定的范围内

In [6]:
input_sequence = input_sequences()
output_neuron = LIFNeuron()
stdp = STDP(num_inputs, 1)
# 初始化 定义输出神经元为1

output_spikes = np.zeros(time_steps)
# 记录输出神经元是否发送脉冲

weights_over_step = np.zeros((time_steps, num_inputs))
# 记录每个时间步的权重 行表示时间步 列表示输入神经元的权重

V_membrane = np.zeros(time_steps)
# 记录输出神经元在买个时间步的膜电位变化

for t in range(time_steps):
    input_spikes = input_sequence[t]
    I_syn = np.dot(stdp.w.T, input_spikes)[0] * 5.0
    # 计算突触电流 用权重和输入神经元的脉冲进行点积 再用放大系数增强信号输入的强度

    output_neuron.update(I_syn)
    # 更新输出神经元状态

    output_spikes[t] = output_neuron.spike
    V_membrane[t] = output_neuron.V
    # 记录当前时间步是否发放脉冲和膜电位

    stdp.update_weights(input_spikes, np.array([output_neuron.spike]))
    # 更新突触权重

    weights_over_step[t] = stdp.w[:, 0]
    # 记录当前时间步的权重

output_spike_times = np.where(output_spikes == 1)[0] * dt
# 找出所有时间步中 输出神经元发送脉冲的时间点

print("神经元发送脉冲的时间点: ", output_spike_times)
print("最终突触权重: ", stdp.w.flatten())

神经元发送脉冲的时间点:  [ 10.  30.  50.  60.  80. 100. 120. 140. 160. 180. 200. 220. 240. 260.
 280. 300. 320. 340. 360. 380. 400. 420. 440. 460. 480. 500. 520. 540.
 560. 580. 600. 620. 640. 660. 680. 700. 710. 730. 750. 770. 790. 810.
 830. 850. 870. 890. 910. 930. 950. 970. 990.]
最终突触权重:  [0.98508063 0.98705937 0.96255351 0.93748825 0.84595857]


In [ ]:
plt.figure(figsize=(10, 8))

plt.subplot(4, 1, 1)
for i in range(num_inputs):
    spike_times = np.where(input_sequence[:, i] == 1)[0] * dt
    plt.scatter(spike_times, np.full_like(spike_times, i), marker='|', color='black')
plt.title('Input Spike Train')
plt.ylabel('Input Neuron Index')
plt.xlim(0, simulation_time)

plt.subplot(4, 1, 2)
plt.plot(np.arange(time_steps) * dt, V_membrane, label='Membrane Potential')
plt.plot(np.arange(time_steps) * dt, np.full(time_steps, V_th), 'r--', label='Threshold')
plt.title('Output Neuron Membrane Potential')
plt.ylabel('Membrane Potential (V)')
plt.xlim(0, simulation_time)
plt.legend()

plt.subplot(4, 1, 3)
spike_times = np.where(output_spikes == 1)[0] * dt
plt.scatter(spike_times, np.zeros_like(spike_times), marker='|', color='red')
plt.title('Output Spike Train')
plt.ylabel('Output Neuron')
plt.xlim(0, simulation_time)

plt.subplot(4, 1, 4)
for i in range(num_inputs):
    plt.plot(np.arange(time_steps) * dt, weights_over_step[:, i], label=f'Weight {i}')
plt.title('Synaptic Weights Over Time')
plt.xlabel('Time (ms)')
plt.ylabel('Weight')
plt.legend()
plt.xlim(0, simulation_time)

plt.tight_layout()
plt.show()